In [ ]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
import numpy as np
print(np.__version__)

1.21.6


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!mkdir /content/drive/MyDrive/SSD_SIBI
!mkdir /content/drive/MyDrive/SSD_SIBI/data

In [ ]:
!curl -L "https://app.roboflow.com/ds/Xm8xzbRAxk?key=ZMSy423z9F" > /content/drive/MyDrive/SSD_SIBI/roboflow.zip;
!unzip /content/drive/MyDrive/SSD_SIBI/roboflow.zip -d /content/drive/MyDrive/SSD_SIBI/data

In [ ]:

!unzip /content/drive/MyDrive/SSD_SIBI/roboflow.zip -d /content/drive/MyDrive/SSD_SIBI/data

Archive:  /content/drive/MyDrive/BISINDO/Bisindo_tfrecord.zip
 extracting: /content/drive/MyDrive/BISINDO/Data/README.dataset.txt  
 extracting: /content/drive/MyDrive/BISINDO/Data/README.roboflow.txt  
   creating: /content/drive/MyDrive/BISINDO/Data/test/
 extracting: /content/drive/MyDrive/BISINDO/Data/test/Letters.tfrecord  
 extracting: /content/drive/MyDrive/BISINDO/Data/test/Letters_label_map.pbtxt  
   creating: /content/drive/MyDrive/BISINDO/Data/train/
 extracting: /content/drive/MyDrive/BISINDO/Data/train/Letters.tfrecord  
 extracting: /content/drive/MyDrive/BISINDO/Data/train/Letters_label_map.pbtxt  
   creating: /content/drive/MyDrive/BISINDO/Data/valid/
 extracting: /content/drive/MyDrive/BISINDO/Data/valid/Letters.tfrecord  
 extracting: /content/drive/MyDrive/BISINDO/Data/valid/Letters_label_map.pbtxt  


In [ ]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = '/content/drive/MyDrive/SSD_SIBI/Data/test/Letters (1).tfrecord'
train_record_fname = '/content/drive/MyDrive/SSD_SIBI/Data/train/Letters (1).tfrecord'
label_map_pbtxt_fname = '/content/drive/MyDrive/SSD_SIBI/Data/train/Letters_label_map (1).pbtxt'

In [ ]:
import sys
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3483, done.
remote: Counting objects: 100% (3483/3483), done.
remote: Compressing objects: 100% (2911/2911), done.
remote: Total 3483 (delta 907), reused 1494 (delta 519), pack-reused 0
Receiving objects: 100% (3483/3483), 46.91 MiB | 30.79 MiB/s, done.
Resolving deltas: 100% (907/907), done.


In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696118 sha256=5ce2d222abe16e926c204d2f3eef88712fd9c1a13848ff06cb5f0d61259ea9c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-qc3w_y6q/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=96f6998f63e1ee99ad563d998dc0250124f7eb079ed1ce93c336cc819f0ae160
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=884df1edc72581c92807c7ed5151a21af8016bf80fecb1ecf61d7008d7645518
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_p

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
##change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'ssd-mobilenet' : {
        'model_name': 'ssd_mobilenet_v2',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
        'batch_size' : 16
    }
    
}

#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'ssd-mobilenet'

num_steps = 40000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [ ]:
#download pretrained weights
%cd /content/drive/MyDrive/SSD_SIBI
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

/content/drive/MyDrive/BISINDO
--2022-08-11 05:53:27--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.119.128, 2a00:1450:4013:c00::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.119.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz.1’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  41.5MB/s    in 0.5s    

2022-08-11 05:53:28 (41.5 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz.1’ saved [20515344/20515344]



In [ ]:
#download base training configuration file
%cd /content/drive/MyDrive/SSD_SIBI
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/content/drive/MyDrive/BISINDO
--2022-08-11 05:53:45--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4684 (4.6K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config’

ssd_mobilenet_v2_fp 100%[===================>]   4.57K  --.-KB/s    in 0.002s  

2022-08-11 05:53:46 (2.31 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config’ saved [4684/4684]



In [ ]:
#prepare
pipeline_fname = '/content/drive/MyDrive/SSD_SIBI/' + base_pipeline_file
fine_tune_checkpoint = '/content/drive/MyDrive/SSD_SIBI/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd /content/drive/MyDrive/SSD_SIBI
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

/content/drive/MyDrive/SSD_SIBI
writing custom configuration file


In [ ]:
%cat /content/drive/MyDrive/SSD_SIBI/pipeline_file.config

# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 26
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator 

In [ ]:
pipeline_file = '/content/drive/MyDrive/SSD_SIBI/pipeline_file.config'
model_dir = '/content/drive/MyDrive/SSD_SIBI/training'

In [ ]:
path_to_ckpt_meta = '/content/drive/MyDrive/SSD_SIBI/training/checkpoint/ckpt-41'

In [ ]:
!pip install -q opencv-python-headless==4.1.2.30

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 11s (40.1 MB/s)
(Reading database ... 155682 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155660 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+

Training Data

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

2022-09-01 01:20:23.645995: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0901 01:20:23.651578 139774546442112 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 40000
I0901 01:20:23.657815 139774546442112 config_util.py:552] Maybe overwriting train_steps: 40000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0901 01:20:23.658001 139774546442112 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0901 01:20:23.683851 139774546442112 deprecation.py:356] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experiment

## Exporting a Trained Inference Graph
Still to come for TF2 models, we will be updating this Colab notebook accordingly as the functionality is added. 

In [ ]:
#see where our model saved weights
%ls '/content/drive/MyDrive/SSD_SIBI/training'

checkpoint                   ckpt-38.index
ckpt-35.data-00000-of-00001  ckpt-39.data-00000-of-00001
ckpt-35.index                ckpt-39.index
ckpt-36.data-00000-of-00001  ckpt-40.data-00000-of-00001
ckpt-36.index                ckpt-40.index
ckpt-37.data-00000-of-00001  ckpt-41.data-00000-of-00001
ckpt-37.index                ckpt-41.index
ckpt-38.data-00000-of-00001  train/


In [ ]:
#run conversion script
import re
import numpy as np

output_directory = '/content/drive/MyDrive/BISINDO/fine_tuned_model'

#place the model weights you would like to export here
last_model_path = '/content/drive/MyDrive/BISINDO/training'
print(last_model_path)
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

/content/drive/MyDrive/BISINDO/training
2022-08-11 06:14:33.647944: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0811 06:14:33.798602 140321778276224 deprecation.py:628] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, ele

In [ ]:
%ls '/content/drive/MyDrive/SSD_SIBI/fine_tuned_model/saved_model/'

ls: cannot access '/content/drive/MyDrive/BISINDO/fine_tuned_model/saved_model/': No such file or directory


In [ ]:
%ls '/content/drive/MyDrive/SSD_SIBI/training/'

ls: cannot access '/content/drive/MyDrive/BISINDO/training/': No such file or directory


In [ ]:
!pip install tf-nightly

In [ ]:
#exporting 
!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
 --pipeline_config_path {pipeline_file} \
 --trained_checkpoint_dir {last_model_path} \
 --output_directory /content/drive/MyDrive/SSD_SIBI/output/exported_models/tflite_inference

2022-08-11 06:25:43.989674: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0811 06:25:55.637080 140610379306880 save_impl.py:72] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7fe1e03d1d50>, because it is not built.
W0811 06:25:55.907278 140610379306880 save_impl.py:72] Skipping full serialization of Keras layer <keras.layers.convolutional.separable_conv2d.SeparableConv2D object at 0x7fe1e029cdd0>, because it is not built.
W0811 06:25:55.907510 140610379306880 save_impl.py:72] Skipping full serialization of Keras layer <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7fe15734f810>, because it is not built.
W0811 06:25:55.907627 140610379306880 save_impl.py:72] Skipping full serialization of Keras layer <keras.layers.core.lambda_layer.Lambda object at 

In [ ]:
pip install tf_nightly

In [ ]:
saved_tflite_inference = "/content/drive/MyDrive/SSD_SIBI/output/exported_models/tflite_inference/saved_model"

In [ ]:
!mkdir /content/drive/MyDrive/SSD_SIBI/output/tflite/

In [ ]:
# The default inference type is Floating-point.
%cd /content/drive/MyDrive/SSD_SIBI/output/

!tflite_convert --saved_model_dir=/content/drive/MyDrive/BISINDO/output/exported_models/tflite_inference/saved_model --output_file=tflite/detect_without_metadata.tflite

/content/drive/MyDrive/BISINDO/output
2022-08-11 07:04:57.883913: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-08-11 07:05:09.081173: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-08-11 07:05:09.081229: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
Estimated count of arithmetic ops: 1.801 G  ops, equivalently 0.900 G  MACs


In [ ]:
tf_lite_model_path_without_metadata = "/content/drive/MyDrive/SSD_SIBI/output/tflite/detect_without_metadata.tflite"

In [ ]:

import tensorflow as tf

saved_model_dir = saved_tflite_inference

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
open(tf_lite_model_path_without_metadata, "wb").write(tflite_model)


11574392

In [ ]:
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tf_lite_model_path_without_metadata )
 
print(interpreter.get_input_details()[0]['shape'])
print(interpreter.get_input_details()[0]['dtype'])

# Print output shape and type
print(interpreter.get_output_details()[0]['shape'])
print(interpreter.get_output_details()[0]['dtype'])

[  1 320 320   3]
<class 'numpy.float32'>
[]
<class 'numpy.float32'>


In [ ]:
pip install tflite-support-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tflite-model-maker 0.4.1 requires flatbuffers==1.12, but you have flatbuffers 2.0 which is incompatible.
tflite-model-maker 0.4.1 requires tf-models-official==2.3.0, but you have tf-models-official 2.9.2 which is incompatible.
tensorflow 2.9.1 requires flatbuffers<2,>=1.12, but you have flatbuffers 2.0 which is incompatible.
scann 1.2.6 requires tensorflow~=2.8.0, but you have tensorflow 2.9.1 which is incompatible.


In [ ]:
''' Writing MetaData to TfLite Model '''

import tflite_support
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata

ObjectDetectorWriter = object_detector.MetadataWriter
_MODEL_PATH = tf_lite_model_path_without_metadata
_LABEL_FILE = "/content/drive/MyDrive/BISINDO/Data/train/labelmap.txt"
_SAVE_TO_PATH = "/content/drive/MyDrive/BISINDO/output/tflite/detect_with_metadata.tflite"

writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5], [_LABEL_FILE])
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

Metadata populated:
{
  "name": "ObjectDetector",
  "description": "Identify which of a known set of objects might be present and provide information about their positions within the given image or a video stream.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be detected.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
    